# PREPROCESSING MESH DATA TO LINKED DATA
In this notebook, we evaluate the mesh assets in a session repository.
For every obj or ply polygonal mesh, a MESHNODE metadata class is created 
that governs all the metadata of the mesh (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) and O3D.GEOMETRY.MESH files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    

#IMPORT MODULES
import Algorithms.linkeddatatools as ld
import Classes.meshnode as mn
import Algorithms.scan2bim as s2b

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name 'self' is not defined

## 1. INITIALIZE SESSION

In [ ]:
## INPUTS
project_path= "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent"
session_path = project_path + "\\week 22"
bim_folderpath = project_path + "\\BIM"
pcd_folderpath = session_path + "\\PCD"
img_folderpath = session_path + "\\IMG"
ortho_folderpath =session_path + "\\ORTHO"
mesh_folderpath = session_path + "\\MESH"

sensor="Holelens" # (string) P30, BLK, Hololens2, CANON (zie exif), etc.
accuracy=0.05
## session initialization missing

## 2. READ PRESENT OBJ/PLY FILES CONTAINING POLYGONAL MESHES
A MeshNode is created per polygonal mesh in the session

In [ ]:
# Retrieve all image files in the session along with their xml metadata
importlib.reload(ld)
importlib.reload(s2b)
importlib.reload(mn)

all_session_file_paths=ld.getListOfFiles(session_path) 
mesh_file_paths=[] # these are string file paths
meshnodelist=[]
obj_counter=0
ply_counter=0

for file_path in all_session_file_paths:        
    if file_path.endswith(".obj") or file_path.endswith(".ply"): 
        #create imgnode
        print('creating MeshNode from '+file_path)
        meshnode=mn.MeshNode()
        meshnode.session_path=session_path
        meshnode.timestamp=ld.get_timestamp(file_path) # this is a long float nr
        meshnode.name=ld.get_filename(file_path)
        meshnode.mesh_path=file_path
        mesh_file_paths.append(file_path) 
        meshnode.sensor = sensor
        meshnode.accuracy=accuracy

        meshnode.o3d_mesh=o3d.io.read_triangle_mesh(file_path)  
        # visualize(meshnode.mesh) # optionally view the mesh node

        if file_path.endswith(".obj"):
            meshnode.set_metadata_from_obj()
            obj_counter+=1
        elif file_path.endswith(".ply"):
            meshnode.set_metadata_from_ply()
            ply_counter+=1

        # create guid if none is present
        if meshnode.guid is None:
            meshnode.guid= '{'+str(uuid.uuid1())+'}'    
       
        meshnodelist.append(meshnode)              
print(str(len(meshnodelist))+' MeshNodes are created.')
print(str(obj_counter)+' are .obj files')
print(str(ply_counter)+' are .ply files')
print('Note that these nodes do not necesserily contain the actual data but only the metadata')